## Importing the packages





In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

## Preprocessing the data

In [ ]:

!wget https://github.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/blob/main/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip


--2023-11-23 10:42:21--  https://github.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/blob/main/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/raw/main/chatbot_nlp.zip [following]
--2023-11-23 10:42:21--  https://github.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/raw/main/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/main/chatbot_nlp.zip [following]
--2023-11-23 10:42:21--  https://raw.githubusercontent.com/Jagruthi-Sarikonda/Generative_Chatbot_LSTM_Implementation/main/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.1

### Reading the data from the files

We analyze every .yaml file, merging sentences as needed. Eliminate undesired data types generated during data parsing. Add <START> and <END> to all responses. Establish a Tokenizer and integrate the entire vocabulary (questions + answers) into it.





In [ ]:

from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 1894



### Preparing data for Seq2Seq model



In [ ]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(564, 22) 22
(564, 74) 74
(564, 74, 1894)


## Defining the Encoder-Decoder model









In [ ]:

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs) # 200-dimensional GloVe embeddings
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 22)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 74)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 22, 200)              378800    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 74, 200)              378800    ['input_2[0][0]']             
                                                                                              

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [ ]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 )
model.save( 'model.h5' )


Epoch 1/150
12/12 [==============================] - 17s 185ms/step - loss: 7.5244
Epoch 2/150
12/12 [==============================] - 3s 224ms/step - loss: 6.7254
Epoch 3/150
12/12 [==============================] - 3s 224ms/step - loss: 5.9840
Epoch 4/150
12/12 [==============================] - 2s 154ms/step - loss: 5.8289
Epoch 5/150
12/12 [==============================] - 1s 123ms/step - loss: 5.7791
Epoch 6/150
12/12 [==============================] - 1s 95ms/step - loss: 5.7500
Epoch 7/150
12/12 [==============================] - 2s 137ms/step - loss: 5.7284
Epoch 8/150
12/12 [==============================] - 1s 78ms/step - loss: 5.7035
Epoch 9/150
12/12 [==============================] - 1s 105ms/step - loss: 5.6729
Epoch 10/150
12/12 [==============================] - 1s 95ms/step - loss: 5.6493
Epoch 11/150
12/12 [==============================] - 2s 162ms/step - loss: 5.6224
Epoch 12/150
12/12 [==============================] - 1s 108ms/step - loss: 5.5796
Epoch 13/150
12

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_output_train, decoder_output_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_output_data, test_size=0.2, random_state=42
)

# Train the model with validation data
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    batch_size=50,
    epochs=150
)

# Save the trained model
model.save('model.h5')

Epoch 1/150
10/10 [==============================] - 5s 503ms/step - loss: 2.9341 - val_loss: 2.9856
Epoch 2/150
10/10 [==============================] - 0s 50ms/step - loss: 2.9166 - val_loss: 3.0294
Epoch 3/150
10/10 [==============================] - 1s 75ms/step - loss: 2.8943 - val_loss: 3.0781
Epoch 4/150
10/10 [==============================] - 1s 73ms/step - loss: 2.8647 - val_loss: 3.1169
Epoch 5/150
10/10 [==============================] - 1s 55ms/step - loss: 2.8634 - val_loss: 3.1366
Epoch 6/150
10/10 [==============================] - 1s 57ms/step - loss: 2.8394 - val_loss: 3.1695
Epoch 7/150
10/10 [==============================] - 1s 58ms/step - loss: 2.8349 - val_loss: 3.2575
Epoch 8/150
10/10 [==============================] - 1s 55ms/step - loss: 2.8193 - val_loss: 3.2296
Epoch 9/150
10/10 [==============================] - 1s 66ms/step - loss: 2.7934 - val_loss: 3.2992
Epoch 10/150
10/10 [==============================] - 1s 53ms/step - loss: 2.7880 - val_loss: 3.275

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [ ]:

def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model


## Talking with our Chatbot

In [ ]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


1. Initially, we input a question to enc_model to predict the state values.
2. The obtained state values are then configured in the decoder's LSTM.
3. Next, we create a sequence that starts with the `<start>` element. This sequence is fed into the dec_model.
4. Subsequently, we replace the `<start>` element with the element predicted by the dec_model and update the state values.
5. These steps are repeated iteratively until we reach the `<end>` tag or reach the maximum answer length.





In [ ]:
enc_model, dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict(str_to_tokens(input('Enter question: ')))

    # Initialize target sequence with the '<start>' token
    empty_target_seq = np.zeros((1, 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']

    stop_condition = False
    decoded_translation = ''

    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)

        # Get the index of the predicted word
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])

        # Convert the index to the corresponding word
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                decoded_translation += ' {}'.format(word)
                sampled_word = word

        # Check for termination conditions
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

        # Append the predicted word index to the target sequence
        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = sampled_word_index

        # Update the states values
        states_values = [h, c]

    print(decoded_translation)
